In [2]:
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel 
from diffusers import StableDiffusionXLControlNetPipeline, AutoencoderKL # controlNet with Stable Diffusion XL
from diffusers import EulerDiscreteScheduler # Import samplers
from diffusers import DPMSolverMultistepScheduler # Import samplers
from diffusers.utils import load_image
from tqdm import tqdm
from prompt import prompt_generator, prompt_generator_no_human
import numpy as np


c:\Users\aaron\Documents\git\controlnet\.myvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


runwayml/stable-diffusion-v1-5
stablediffusionapi/realistic-vision


In [3]:
# ControlNet model
controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_scribble", torch_dtype=torch.float16)

# Define stable diffusion pipeline with controlnet
# We use the realistic-vision-v20-2047 model for this example (finetunned on realistic images of people)
pipe = StableDiffusionControlNetPipeline.from_pretrained("stablediffusionapi/realistic-vision-v20-2047", controlnet=controlnet, safety_checker=None, torch_dtype=torch.float16)
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)

# Enable efficient implementations using xformers for faster inference
pipe.enable_xformers_memory_efficient_attention()
pipe.enable_model_cpu_offload()

unet\diffusion_pytorch_model.safetensors not found
Loading pipeline components...:  67%|██████▋   | 4/6 [00:01<00:00,  3.26it/s]c:\Users\aaron\Documents\git\controlnet\.myvenv\Lib\site-packages\transformers\models\clip\feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
Loading pipeline components...: 100%|██████████| 6/6 [00:05<00:00,  1.13it/s]
You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve anal

In [4]:
prompt = prompt_generator_no_human()
print(prompt)

A medium shot view of a colorful cardboard box with yellow tape from the Zara webshop, in front of a city center background, the weather is sunny, high photorealistic quality.


In [6]:
# Load wireframe image
image_input = load_image("wireframes\\5_1.jpg")

# Prompt
prompt = prompt_generator_no_human()

# Run the pipeline
image_output = pipe(prompt=prompt, negative_prompt="poor quality, cartoon, mulitiple boxes, cardboard in the background", image=image_input, num_inference_steps=30).images[0]

# Save the output
image_output.save(f"test.jpg")
print("Save test.jpg!")

  0%|          | 0/30 [00:01<?, ?it/s]


TypeError: argument of type 'NoneType' is not iterable

XLSD

In [7]:
# initialize the models and pipeline

controlnet_conditioning_scale = 0.5  # recommended for good generalization
controlnet = ControlNetModel.from_pretrained(

    "thingthatis/controlnet-canny-sdxl-1.0", torch_dtype=torch.float16

)

vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)

pipe = StableDiffusionXLControlNetPipeline.from_pretrained(

    "stabilityai/stable-diffusion-xl-base-1.0", controlnet=controlnet, vae=vae, torch_dtype=torch.float16

)

pipe.enable_model_cpu_offload()

Loading pipeline components...: 100%|██████████| 7/7 [00:29<00:00,  4.17s/it]


In [10]:
image_input = load_image("wireframes\\5_1.jpg")

# Prompt
prompt = prompt_generator_no_human()

# Run the pipeline
image_output = pipe(prompt=prompt, image=image_input, num_inference_steps=30).images[0]

# Save the output
image_output.save(f"test.jpg")
print("Save test.jpg!")

  0%|          | 0/30 [00:00<?, ?it/s]c:\Users\aaron\Documents\git\controlnet\.myvenv\Lib\site-packages\diffusers\models\attention_processor.py:1279: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  hidden_states = F.scaled_dot_product_attention(
100%|██████████| 30/30 [12:59<00:00, 25.99s/it]


Save test.jpg!
